In [17]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from gensim import corpora, models, similarities
from gensim import corpora
from gensim.models import word2vec
import gensim
import pandas as pd
import numpy as np
import os
from glob import glob
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import sys
from sklearn import linear_model
sys.path.append(os.getcwd())
import matplotlib.pyplot as plt
os.chdir('/Users/xiaodiu/Documents/github/JudgeSentences/train/')
from train_ngrams import textfeature
import numpy as np
%matplotlib inline
import re
from keras.models import model_from_json

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model
from keras import initializations

from keras import backend as K
from keras.engine.topology import Layer, InputSpec


from nltk import tokenize
#import tensorflow as tf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectKBest, chi2, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import hstack, vstack

## test on text features class

In [ ]:
ngrams = textfeature()

In [ ]:
## load data
train_data = ngrams.load_data('../holger_train_judgeyear.csv',index_col=0)
test_data = ngrams.load_data('../holger_test_judgeyear.csv', index_col=0)


In [ ]:
judge_year_index = ngrams.load_data('../datasets/judge_year2index.pkl', format = 'pkl')
ngram_dict = ngrams.load_data('../datasets/grams_dict2002-2016/grams_dict.pkl', format = 'pkl')

bow_feature = ngrams.load_data('../datasets/grams_dict2002-2016/bow_features.pkl', format = 'pkl')
bi_feature = ngrams.load_data('../datasets/grams_dict2002-2016/2grams_feature.pkl', format = 'pkl')
tri_feature = ngrams.load_data('../datasets/grams_dict2002-2016/3grams_feature.pkl', format = 'pkl')
for_feature = ngrams.load_data('../datasets/grams_dict2002-2016/4grams_feature.pkl', format = 'pkl')
fiv_feature = ngrams.load_data('../datasets/grams_dict2002-2016/5grams_feature.pkl', format = 'pkl')

In [ ]:
features = bow_feature

In [ ]:
ngrams.process_data(train_data, judge_year_index, features, istrain = True, 
                    drops= ['judgeid', 'disparity_race','sentyr'],y_label = 'disparity_race')
ngrams.process_data(test_data, judge_year_index, features, istrain = False,
                    drops= ['judgeid', 'disparity_race','sentyr'],y_label = 'disparity_race')

In [ ]:
ngrams.get_train_test(features)
bow_train, bow_test = ngrams.get_vector()
X_train, X_test = ngrams.get_tfidf(bow_train, bow_test)
train_total, test_total = ngrams.combine_data(X_train, X_test)

In [ ]:
regr_bow = linear_model.LinearRegression()
y_pre = ngrams.model_pre(regr_bow, train_total, test_total)

## plot
ngrams.plot_scatter(y_pre, name = 'bi_gram_reg_disparity_race.png')

In [ ]:
## trainning on USSC_dataset

In [ ]:
USSC_train_data = pd.read_csv('../Trac_USSC_train_judgeyear2.csv', index_col=0)
USSC_test_data  = pd.read_csv('../Trac_USSC_test_judgeyear2.csv', index_col=0)

In [ ]:
### get the ngrams for ussc dataset

In [ ]:
model_zoo = Counter()
model_zoo['OLS'] = linear_model.LinearRegression()
model_zoo['PLS'] = cross_decomposition.PLSRegression(n = 200)
model_zoo['RF']  = RandomForestRegressor()
model_zoo['Elastic Net'] = linear_model.ElasticNet(alpha=0.1, l1_ratio=0.7)

In [ ]:
ngrams.run_model(train_data, test_data, judge_year_index, features, 
                 model_zoo=model_zoo, plot_name = 'bow_features',
                 drops = ['judgeid', 'demean_harshness','sentyr'],
                 y_label = 'demean_harshness', 
                 dataset = 'Holger', model_name = 'OLS')

## Train LSTM Attension model

In [ ]:
judge_year_case = pd.read_pickle('../datasets/judge2year_case.pkl')
opinion2floder = pd.read_pickle('../datasets/opinion2floder.pkl')
judges = judge_year_case.keys()
data_train = train_data
data_test = test_data

In [ ]:

y_train = []
y_test = []
texts = []
max_year = 0
max_judge = 0
reviews = Counter()

In [4]:
EMBEDDING_DIM = 100
MAX_SENTS = 2000
MAX_SENT_LEN = 100
MAX_NB_WORDS = 500000

In [ ]:
year_range = (2002,2011)
for i, judge in enumerate(judges):
    opinions = judge_year_case[judge]
    review_year = Counter()
    for year in opinions.keys():
        review_all = []
        if year < year_range[0] or year > year_range[1]:
            continue
        opilist = opinions[year]
        for opi in opilist:    
            fold = opinion2floder[str(opi)]
            filepath = '../rawdata/text/' + fold + '/' + str(opi) + '.txt'
            #print(filepath)
            text = open(filepath).read()    
            normtext = text.strip().lower()
            sentences = tokenize.sent_tokenize(normtext)
            for sent in sentences:
                if len(sent.split(' ')) > 5:
                    sent = re.sub('[^a-z0-9]',' ',sent)
                    review_all.append(sent)
                    texts.append(sent)
        review_year[year] = review_all
    reviews[judge] = review_year
    print(i)

In [ ]:
pd.to_pickle(reviews, '../datasets/lstmdata/all_reviews.pkl')
pd.to_pickle(texts, '../datasets/lstmdata/textsfortoken.pkl')

In [ ]:
train_len = 0
for i in train_data.index:
    senty = train_data.sentyr[i]
    judge = train_data.judgeid[i]
    try :
        opinions = reviews[judge][senty]
        if opinions == 0:
            print('no opnions this %s for %s' %(senty, judge))
            continue
        train_len += 1
    except KeyError :
        print('no opnions this %s for %s' %(senty, judge))
    
test_len = 0
for i in test_data.index:
    senty = test_data.sentyr[i]
    judge = test_data.judgeid[i]
    try :
        opinions = reviews[judge][senty]
        if opinions == 0:
            print('no opnions this %s for %s' %(senty, judge))
            continue
        test_len += 1
    except KeyError :
        print('no opnions this %s for %s' %(senty, judge))   
        

In [ ]:
lstm_train = np.zeros((train_len,MAX_SENTS, MAX_SENT_LEN), dtype='int32')
lstm_test = np.zeros((test_len,MAX_SENTS, MAX_SENT_LEN), dtype='int32')

In [ ]:
num = 0
for i in train_data.index:
    senty = train_data.sentyr[i]
    judge = train_data.judgeid[i]
    try :
        opinions = reviews[judge][senty]
        if opinions == 0:
            print('no opnions this %s for %s' %(senty, judge))
            continue
        y_train.append(train_data.demean_harshness[i])
        for j, sent in enumerate(opinions):
            if j< MAX_SENTS:
                sent = re.sub('[^a-z0-9]',' ',sent)
                wordTokens = text_to_word_sequence(sent)
                k=0
                for _, word in enumerate(wordTokens):
                    if k<MAX_SENT_LEN:
                        index = tokenizer.word_index[word]
                        if index == 0:
                            continue
                        lstm_train[num,j,k] = index
                        #lstm_train[num,j,k] = tokenizer.word_index[word]
                        k=k+1  
        num += 1
    except KeyError :
        print('no opnions this %s for %s' %(senty, judge)) 

num = 0
for i in test_data.index:
    senty = test_data.sentyr[i]
    judge = test_data.judgeid[i]
    try :
        opinions = reviews[judge][senty]
        if opinions == 0:
            print('no opnions this %s for %s' %(senty, judge))
            continue
        y_test.append(test_data.demean_harshness[i])
        for j, sent in enumerate(opinions):
            if j< MAX_SENTS:
                sent = re.sub('[^a-z0-9]',' ',sent)
                wordTokens = text_to_word_sequence(sent)
                k=0
                for _, word in enumerate(wordTokens):
                    if k<MAX_SENT_LEN :
                        index = tokenizer.word_index[word]
                        if index == 0:
                            continue
                        lstm_test[num,j,k] = index
                        k=k+1 
        num += 1
    except KeyError :
        print('no opnions this %s for %s' %(senty, judge)) 

In [ ]:
np.save("../datasets/lstmdata/lstm_train.npy", lstm_train)
np.save("../datasets/lstmdata/lstm_test.npy", lstm_test)
np.save("../datasets/lstmdata/y_train.npy", y_train)
np.save("../datasets/lstmdata/y_test.npy", y_test)

In [6]:
lstm_train = np.load("../datasets/lstmdata/lstm_train.npy")
lstm_test = np.load("../datasets/lstmdata/lstm_test.npy")
y_train = np.load("../datasets/lstmdata/y_train.npy")
y_test = np.load("../datasets/lstmdata/y_test.npy")

In [13]:
texts = pd.read_pickle('../datasets/lstmdata/textsfortoken.pkl')
tokenizer = Tokenizer(nb_words=None)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index

In [14]:
model = word2vec.Word2Vec.load('../datasets/word2vec.model')
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        embedding_vector = model.wv[word]
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    except KeyError as err:
        embedding_matrix[i] = np.zeros(EMBEDDING_DIM)
        
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LEN,
                            trainable=True)

In [15]:
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        eij = K.tanh(K.dot(x, self.W))
        
        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [ ]:
sentence_input = Input(shape=(MAX_SENT_LEN,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_dense = TimeDistributed(Dense(200))(l_lstm)
l_att = AttLayer()(l_dense)
sentEncoder = Model(sentence_input, l_att)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LEN), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
l_dense_sent = TimeDistributed(Dense(200))(l_lstm_sent)
l_att_sent = AttLayer()(l_dense_sent)
preds = Dense(1, activation=None)(l_att_sent)
model = Model(review_input, preds)

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['acc'])

print("model fitting - Hierachical attention network")
history =model.fit(lstm_train, y_train, validation_data=(lstm_test, y_test),
        nb_epoch = 6000, batch_size=1)

model_json = model.to_json()
with open("../datasets/lstmdata/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("../datasets/lstmdata/model.h5")

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

model fitting - Hierachical attention network
Train on 3423 samples, validate on 837 samples
Epoch 1/6000
2402/3423 [====================>.........] - ETA: 14479s - loss: 0.0238 - acc: 0.0037    